<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cpp_bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

qita 
ビット演算 (bit 演算) の使い方を総特集！ 〜 マスクビットから bit DP まで 〜 by 大槻 (通称、けんちょん) 

https://qiita.com/drken/items/7c6ff2aa4d8fce1c9361

全探索について  
https://drken1215.hatenablog.com/entry/2019/12/14/171657

問題集  
https://algo-method.com/courses/3


In [ ]:
# 問題 整数を bitset で表せ
# bitset は桁数指定が必要 bitset<8> とか
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    int a = 45; // 実際上は 0b101101、あるいは 0x2d と書く
    cout << (bitset<6>) a << endl; //=> 001011101
    cout << (bitset<6>) 45 << endl; //=> 001011101
    cout << (bitset<6>) 0x2d << endl; //=> 001011101
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

101101
101101
101101


In [ ]:
# 整数の bitset 演算
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    int a = 45;
    int b = 25;
    cout << a << " AND " << b << " = " << (a&b) << endl; // & 1つだと bit 演算になる
    bitset<8> c(45);
    bitset<8> d(25);
    cout << c << " AND " << d << " = " << (c&d) << endl;
    cout << (bitset<8>)a <<  " AND " << (bitset<8>)b << " = " <<  (bitset<8>)(a&b) << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

45 AND 25 = 9
00101101 AND 00011001 = 00001001
00101101 AND 00011001 = 00001001


In [ ]:
# bitset を整数に変換するには bitset の to_ullong 関数を使う
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main () {
    cout << 0b101101 << endl;
    bitset<8> a = 45;
    bitset<8> b(26);
    cout << a.to_ullong() << ", " << b.to_ullong() << endl;

    // to_string は使えるか => 使えない
    cout << to_string(0b101101) << endl; // 45
    // cout << to_string(a) << endl; // エラー
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

45
45, 26
45


In [ ]:
# 45 OR 25 = 61
# bit 演算の `|` の例
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
int main() {
    int a = 45; 
    int b = 25;
    cout << a << " OR " << b << " = " << (a|b) << endl;
    cout << (bitset<8>)a << " OR " << (bitset<8>)b << " = " << (bitset<8>)(a|b) << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

45 OR 25 = 61
00101101 OR 00011001 = 00111101


In [ ]:
%%html
<style>
    tr td:nth-child(2) {
        color: red;
        text-align: center;
    }
</style>
<table border="1">
<thead>
<tr><th>ビット演算</th><th>演算子</th><th>意味</th></tr>
</thead>
<tbody>
<tr><td>ビット毎のAND演算</td><td>&</td><td>各ビットについて「両方のビットが1ならば1」という操作を適用する。</td></tr>
<tr><td>ビット毎のOR演算</td><td>|</td><td>各ビットについて「少なくとも一方のビットが1ならば1」という操作を適用する。</td></tr>
<tr><td>ビット毎のXOR演算</td><td>^</td><td>各ビットについて「どちらか一方だけが1ならば1」という操作を適用する。</td></tr>
<tr><td>ビット毎のNOT演算</td><td>~</td><td>各ビットについて「ビットを反転する」という操作を適用する。</td></tr>
<tr><td>左シフト演算</td><td><<</td><td>指定したビット数だけビット列を左にずらす。範囲外のビットは切り捨てられ、足りないビットは0で埋められる。</td></tr>
<tr><td>右シフト演算</td><td>>></td><td>指定したビット数だけビット列を右にずらす。範囲外のビットは切り捨てられ、足りないビットは0で埋められる。</td></tr>
</tbody>
</table>

ビット演算,演算子,意味
ビット毎のAND演算,&,各ビットについて「両方のビットが1ならば1」という操作を適用する。
ビット毎のOR演算,|,各ビットについて「少なくとも一方のビットが1ならば1」という操作を適用する。
ビット毎のXOR演算,^,各ビットについて「どちらか一方だけが1ならば1」という操作を適用する。
ビット毎のNOT演算,~,各ビットについて「ビットを反転する」という操作を適用する。
左シフト演算,<<,指定したビット数だけビット列を左にずらす。範囲外のビットは切り捨てられ、足りないビットは0で埋められる。
右シフト演算,>>,指定したビット数だけビット列を右にずらす。範囲外のビットは切り捨てられ、足りないビットは0で埋められる。


In [ ]:
# ビット毎のNOT演算	~	各ビットについて「ビットを反転する」という操作を適用する。
# をやってみる
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
int main() {
    int a = 45;
    bitset<8> b(a);
    cout << a << " : " << b << endl;
    cout <<  "NOT " << b << " = "<<  ~b << endl;
    cout << ~b << " = " << (~b).to_ullong() << endl; // 2+16+64+128 = 210
}

Writing temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

45 : 00101101
NOT 00101101 = 11010010
11010010 = 210


In [ ]:
# 3. ビット演算を用いたフラグ管理
# ここからビットシフト演算子が登場します！
# a 番目のフラグが立っている状態は (1<<a) と表せます。
#さらに、a 番目と b 番目と c 番目のフラグが立っている状態は (1<<a) | (1<<b) | (1<<c) と表せます。
# これを用いて以下のようなフラグ管理ができます！
%%html
<table border="1" style="width: 90%">
<thead>
  <tr>
    <th>やりたいこと</th>
    <th>実装</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>ビット bit に i 番目のフラグが立っているかどうか</td>
    <td>if (bit &amp; (1&lt;&lt;i))</td>
  </tr>
  <tr>
    <td>ビット bit に i 番目のフラグが消えているかどうか</td>
    <td>if (!(bit &amp; (1&lt;&lt;i)))</td>
  </tr>
  <tr>
    <td>ビット bit に i 番目のフラグを立てる</td>
    <td>bit｜= (1&lt;&lt;i)</td>
  </tr>
  <tr>
    <td>ビット bit に i 番目のフラグを消す</td>
    <td>bit &amp;= ~(1&lt;&lt;i)</td>
  </tr>
  <tr>
    <td>ビット bit に何個のフラグが立っているか</td>
    <td>__builtin_popcount(bit)</td>
  </tr>
  <tr>
    <td>ビット bit に i 番目のフラグを立てたもの</td>
    <td>bit｜(1&lt;&lt;i)</td>
  </tr>
  <tr>
    <td>ビット bit に i 番目のフラグを消したもの</td>
    <td>bit &amp; ~(1&lt;&lt;i)</td>
  </tr>
</tbody>
</table>

やりたいこと,実装
ビット bit に i 番目のフラグが立っているかどうか,if (bit & (1<<i))
ビット bit に i 番目のフラグが消えているかどうか,if (!(bit & (1<<i)))
ビット bit に i 番目のフラグを立てる,bit｜= (1<<i)
ビット bit に i 番目のフラグを消す,bit &= ~(1<<i)
ビット bit に何個のフラグが立っているか,__builtin_popcount(bit)
ビット bit に i 番目のフラグを立てたもの,bit｜(1<<i)
ビット bit に i 番目のフラグを消したもの,bit & ~(1<<i)


In [ ]:
# 例えばビット bit に i 番目のフラグが立っているかどうかは if ((bit>>i) & 1) と書くこともできます。
# また、bit｜(1<<i) といった書き方をしましたが、
# (1<<i) のようなものは予め define してしまうことも多いです 
# define BIT_FLAG_0 (1<<0) // 0000 0000 0000 0001
# (const unsigned int を使えという意見もあります)
# __builtin_popcount() については GCC のビルトイン関数ですので、GCC で用いる必要があります。

# 以下にフラグ管理の具体的な動作確認を示します (ビットは const unsigned int で表しています):

%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

const unsigned int BIT_FLAG_0 = (1 << 0); // 0000 0000 0000 0001
const unsigned int BIT_FLAG_1 = (1 << 1); // 0000 0000 0000 0010
const unsigned int BIT_FLAG_2 = (1 << 2); // 0000 0000 0000 0100
const unsigned int BIT_FLAG_3 = (1 << 3); // 0000 0000 0000 1000
const unsigned int BIT_FLAG_4 = (1 << 4); // 0000 0000 0001 0000
const unsigned int BIT_FLAG_5 = (1 << 5); // 0000 0000 0010 0000
const unsigned int BIT_FLAG_6 = (1 << 6); // 0000 0000 0100 0000
const unsigned int BIT_FLAG_7 = (1 << 7); // 0000 0000 1000 0000

int main() {
    // {1, 3, 5} にフラグの立ったビット
    unsigned int bit = BIT_FLAG_1 | BIT_FLAG_3 | BIT_FLAG_5;
    cout << "{1, 3, 5} = " << bitset<8>(bit) << endl;

    // ビット {1, 3, 5} について、3 番目のフラグが立っていること
    if (bit & BIT_FLAG_3) {
        cout << "3 is in     " << bitset<8>(bit) << endl;
    }

    // ビット {1, 3, 5} について、0 番目のフラグが立っていないこと
    if (!(bit & BIT_FLAG_0)) {
        cout << "0 is not in " << bitset<8>(bit) << endl;
    }

    // 新たなビット
    unsigned int bit2 = BIT_FLAG_0 | BIT_FLAG_3 | BIT_FLAG_4; // {0, 3, 4}
    cout << bitset<8>(bit) << " AND " << bitset<8>(bit2) << " = " << bitset<8>(bit & bit2) << endl; // {1, 3, 5} AND {0, 3, 4} = {3}
    cout << bitset<8>(bit) << " OR  " << bitset<8>(bit2) << " = " << bitset<8>(bit | bit2) << endl; // {1, 3, 5} OR {0, 3, 4} = {0, 1, 3, 4, 5}

    // bit に 6 番目のフラグを立てる
    cout << "before: " << bitset<8>(bit) << endl;
    bit |= BIT_FLAG_6;
    cout << "after : " << bitset<8>(bit) << " (6 included)" << endl;

    // bit2 から 3 番目のフラグを消す
    cout << "before: " << bitset<8>(bit2) << endl;
    bit2 &= ~BIT_FLAG_3;
    cout << "after : " << bitset<8>(bit2) << " (3 excluded)" << endl;

    return 0;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

{1, 3, 5} = 00101010
3 is in     00101010
0 is not in 00101010
00101010 AND 00011001 = 00001000
00101010 OR  00011001 = 00111011
before: 00101010
after : 01101010 (6 included)
before: 00011001
after : 00010001 (3 excluded)


In [ ]:
# bitset 関数
%%html
<table border="1" style="width: 90%">
<thead>
<tr><th>操作</th><th>書き方</th><th style="width: 50%">備考</th></tr>
</thead>
<tbody>
<tr><td>全てのビットを1にする</td><td>変数.set();</td><td></td></tr>
<tr><td>特定のビットを1にする</td><td>変数.set(1にする位置);</td><td>1にするビットの位置を0始まりのインデックスで指定します。</td></tr>
<tr><td>特定のビットの値を変更する</td><td>変数.set(位置, 値);</td><td>変更するビットの位置を0始まりのインデックスで指定します。値は0か1を指定します。</td></tr>
<tr><td>全てのビットを0にする</td><td>変数.reset();</td><td></td></tr>
<tr><td>特定のビットを0にする</td><td>変数.reset(0にする位置);</td><td>0にするビットの位置を0始まりのインデックスで指定します。</td></tr>
<tr><td>全てのビットを反転する</td><td>変数.flip();</td><td></td></tr>
<tr><td>特定のビットを反転する</td><td>変数.flip(反転する位置);</td><td>反転するビットの位置を0始まりのインデックスで指定します。</td></tr>
<tr><td>特定のビットが1になっているかを調べる</td><td>変数.test(調べる位置);</td><td>調べるビットの位置を0始まりのインデックスで指定します。ビットが1ならtrueを、ビットが0ならfalseを返します。</td></tr>
<tr><td>全てのビットが1になっているかを判定する</td><td>変数.all()</td><td>全てのビットが1ならtrueを、そうでなければfalseを返します。</td></tr>
<tr><td>いずれかのビットが1になっているかを判定する</td><td>変数.any()</td><td>1のビットが存在するならtrueを、そうでなければfalseを返します。</td></tr>
<tr><td>1のビットの個数を数える</td><td>変数.count()</td><td></td></tr>
<tr><td>ビット列を出力する</td><td>cout << 変数;</td><td></td></tr>
<tr><td>ビット列を文字列化する</td><td>変数.to_string()</td><td></td></tr>
<tr><td>ビットに対するアクセス</td><td>変数[位置]</td><td>基本的にはtest、set/resetと同等ですが、範囲外の位置を指定した場合にエラーにならないことに注意する必要があります。</td></tr>
</tbody>
</table>

操作,書き方,備考
全てのビットを1にする,変数.set();,
特定のビットを1にする,変数.set(1にする位置);,1にするビットの位置を0始まりのインデックスで指定します。
特定のビットの値を変更する,"変数.set(位置, 値);",変更するビットの位置を0始まりのインデックスで指定します。値は0か1を指定します。
全てのビットを0にする,変数.reset();,
特定のビットを0にする,変数.reset(0にする位置);,0にするビットの位置を0始まりのインデックスで指定します。
全てのビットを反転する,変数.flip();,
特定のビットを反転する,変数.flip(反転する位置);,反転するビットの位置を0始まりのインデックスで指定します。
特定のビットが1になっているかを調べる,変数.test(調べる位置);,調べるビットの位置を0始まりのインデックスで指定します。ビットが1ならtrueを、ビットが0ならfalseを返します。
全てのビットが1になっているかを判定する,変数.all(),全てのビットが1ならtrueを、そうでなければfalseを返します。
いずれかのビットが1になっているかを判定する,変数.any(),1のビットが存在するならtrueを、そうでなければfalseを返します。


In [ ]:
# bitset 関数を使って書き直す
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    bitset<8> bit;
    // {1, 3, 5} にフラグの立ったビット
    // bit.set(1);bit.set(3);bit.set(5);
    bit.set(1).set(3).set(5);
    cout << "{1, 3, 5} = " << bit << endl;

    // ビット {1, 3, 5} について、3 番目のフラグが立っていること
    if (bit.test(3)) {
        cout << "3 is in     " << bitset<8>(bit) << endl;
    }

    // ビット {1, 3, 5} について、0 番目のフラグが立っていないこと
    if (!(bit.test(0))) {
        cout << "0 is not in " << bitset<8>(bit) << endl;
    }

    // 新たなビット
    // unsigned int bit2 = BIT_FLAG_0 | BIT_FLAG_3 | BIT_FLAG_4; // {0, 3, 4}
    bitset<8> bit2;
    bit2.set(0).set(3).set(4); // {0, 3, 4}
    cout << (bit) << " AND " << (bit2) << " = " << (bit & bit2) << endl; // {1, 3, 5} AND {0, 3, 4} = {3}
    cout << (bit) << " OR  " << (bit2) << " = " << (bit | bit2) << endl; // {1, 3, 5} OR {0, 3, 4} = {0, 1, 3, 4, 5}

    // bit に 6 番目のフラグを立てる
    cout << "before: " << (bit) << endl;
    bit.set(6);
    cout << "after : " << (bit) << " (6 included)" << endl;

    // bit2 から 3 番目のフラグを消す
    cout << "before: " << (bit2) << endl;
    bit2.reset(3);
    cout << "after : " << (bit2) << " (3 excluded)" << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

{1, 3, 5} = 00101010
3 is in     00101010
0 is not in 00101010
00101010 AND 00011001 = 00001000
00101010 OR  00011001 = 00111011
before: 00101010
after : 01101010 (6 included)
before: 00011001
after : 00010001 (3 excluded)


In [ ]:
# マスクビット
# ビットの使い方として最も多いものの 1 つがマスクビットです。
# なじみの深いもので言えば、IP アドレスのマスクビットや割り込み制御のマスクビットなどが挙げられます。
# 多種多様な場面で使用されるマスクビットですが、基本的なアイディアは共通で、
# 
# 複数のフラグをまとめて立てる
# 複数のフラグをまとめて消す
# 必要な情報だけを取り出すために、マスクした部分の情報のみを取り出す
# といったものを効率良く実現するものです。現在のフラグ状態を表すビットを bit として、
# マスクビットを mask としたとき
%%html
<table border="1">
<thead>
<tr><th>やりたいこと</th><th>実装</th></tr>
</thead>
<tbody>
<tr><td>mask で表された部分のフラグをまとめて立てる</td><td>bit ｜= mask</td></tr>
<tr><td>mask で表された部分のフラグをまとめて消す</td><td>bit &= ~mask</td></tr>
<tr><td>mask で表された部分の情報のみを取り出したもの</td><td>bit & mask</td></tr>
<tr><td>mask で表された部分のどれかのフラグが立っているかどうか</td><td>if (bit & mask)</td></tr>
<tr><td>mask で表された部分のすべてのフラグが立っているかどうか</td><td>if ((bit & mask) == mask)</td></tr>
</tbody>
</table>

やりたいこと,実装
mask で表された部分のフラグをまとめて立てる,bit ｜= mask
mask で表された部分のフラグをまとめて消す,bit &= ~mask
mask で表された部分の情報のみを取り出したもの,bit & mask
mask で表された部分のどれかのフラグが立っているかどうか,if (bit & mask)
mask で表された部分のすべてのフラグが立っているかどうか,if ((bit & mask) == mask)


In [ ]:
# 簡単な例を以下に示します。「毒」「麻痺」状態を「回復の薬」で回復させたりしてみます。
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

const unsigned int BIT_FLAG_DAMAGE = (1 << 0); // HP が満タンでないか
const unsigned int BIT_FLAG_DOKU = (1 << 1); // 毒状態になっているか
const unsigned int BIT_FLAG_MAHI = (1 << 2); // 麻痺状態になっているか
const unsigned int BIT_FLAG_SENTOFUNO = (1 << 3); // 戦闘不能状態になっているか

// atack して、単にダメージを受ける
const unsigned int MASK_ATTACK = BIT_FLAG_DAMAGE;

// punch して、ダメージも受けて、麻痺もする
const unsigned int MASK_PUNCH = BIT_FLAG_DAMAGE | BIT_FLAG_MAHI;

// defeat して、相手の HP を 0 にする
const unsigned int MASK_DEFEAT = BIT_FLAG_DAMAGE | BIT_FLAG_SENTOFUNO;

// 「毒」と「麻痺」を回復させる: ~MASK_DOKU_MAHI をかけることで回復
const unsigned int MASK_DOKU_MAHI = BIT_FLAG_DOKU | BIT_FLAG_MAHI;

int main() {
    // start: 0000, 初期状態
    unsigned int status = 0;
    cout << "start: " << bitset<4>(status) << endl;

    // attacked: 0001 になる
    status |= MASK_ATTACK;
    cout << "attacked: " << bitset<4>(status) << endl;

    // punched: 0101 になる, HPはすでに満タンでないので、BIT_FLAG_DAMAGE の部分は変化なし
    status |= MASK_PUNCH;
    cout << "punched: " << bitset<4>(status) << endl;

    // 「毒」または「麻痺」かどうかを判定する
    if (status & MASK_DOKU_MAHI) {
        cout << "You are doku or mahi." << endl;
    }

    // kaihuku: 0001 にする, HPは回復しない、麻痺は回復する
    status &= ~MASK_DOKU_MAHI;
    cout << "kaihuku: " << bitset<4>(status) << endl;

    // defeat: 1001 にする, 戦闘不能にする
    status |= MASK_DEFEAT;
    cout << "sentofuno: " << bitset<4>(status) << endl;

    // kaihuku: 1001 のまま、戦闘不能状態は回復しない
    status &= ~MASK_DOKU_MAHI;
    cout << "sentofuno no mama: " << bitset<4>(status) << endl;

    return 0;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

start: 0000
attacked: 0001
punched: 0101
You are doku or mahi.
kaihuku: 0001
sentofuno: 1001
sentofuno no mama: 1001


In [ ]:
# ちょっと実験
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
int main(){
    cout << bitset<4>(1 << 3) << endl;
    bitset<4> x("0010");
    cout << x << endl;
    cout << x.reset().set(3) << endl;
    x ^= bitset<4>("1111");
    cout << x << endl;
}

In [ ]:
!g++ temp.cpp; ./a.out

1000
0010
1000
0111


In [ ]:
# bitset を使って書き直してみよう
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

const bitset<4> BIT_FLAG_DAMAGE("0001"); // HP が満タンでないか
const bitset<4> BIT_FLAG_DOKU("0010"); // 毒状態になっているか
const bitset<4> BIT_FLAG_MAHI("0100"); // 麻痺状態になっているか
const bitset<4> BIT_FLAG_SENTOFUNO("1000"); // 戦闘不能状態になっているか

// atack して、単にダメージを受ける
bitset<4> MASK_ATTACK = BIT_FLAG_DAMAGE;

// punch して、ダメージも受けて、麻痺もする
bitset<4> MASK_PUNCH = BIT_FLAG_DAMAGE | BIT_FLAG_MAHI;

// defeat して、相手の HP を 0 にする
bitset<4> MASK_DEFEAT = BIT_FLAG_DAMAGE | BIT_FLAG_SENTOFUNO;

// 「毒」と「麻痺」を回復させる: ~MASK_DOKU_MAHI をかけることで回復
bitset<4> MASK_DOKU_MAHI = BIT_FLAG_DOKU | BIT_FLAG_MAHI;

int main() {
    // start: 0000, 初期状態
    bitset<4> status("0000");
    cout << "start: " << status << endl;

    // attacked: 0001 になる
    status |= MASK_ATTACK;
    cout << "attacked: " << status << endl;

    // punched: 0101 になる, HPはすでに満タンでないので、BIT_FLAG_DAMAGE の部分は変化なし
    status |= MASK_PUNCH;
    cout << "punched: " << status << endl;

    // 「毒」または「麻痺」かどうかを判定する
    if ((status & MASK_DOKU_MAHI).any()) {
        cout << "You are doku or mahi." << endl;
    }

    // kaihuku: 0001 にする, HPは回復しない、麻痺は回復する
    status &= ~MASK_DOKU_MAHI;
    cout << "kaihuku: " << status << endl;

    // defeat: 1001 にする, 戦闘不能にする
    status |= MASK_DEFEAT;
    cout << "sentofuno: " << status << endl;

    // kaihuku: 1001 のまま、戦闘不能状態は回復しない
    status &= ~MASK_DOKU_MAHI;
    cout << "sentofuno no mama: " << status << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

start: 0000
attacked: 0001
punched: 0101
You are doku or mahi.
kaihuku: 0001
sentofuno: 1001
sentofuno no mama: 1001


In [ ]:
# メモ
# any の使い方
# // 「毒」または「麻痺」かどうかを判定する
# if ((status & MASK_DOKU_MAHI).any()) {
    # cout << "You are doku or mahi." << endl;
# }

In [ ]:
# 5. ビット演算は集合演算
# 数学の人向けです。ここからは内容が少しずつ難しくなっていきます。
# ビット演算によるフラグ管理の表を集合の言葉で記述してみます:
%%html
<table border="1">
<thead>
  <tr>
    <th>やりたいこと</th>
    <th>実装</th>
    <th>集合で言うと</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>ビット S に i 番目のフラグが立っているかどうか</td>
    <td>if (S &amp; (1&lt;&lt;i))</td>
    <td>i∈S かどうか</td>
  </tr>
  <tr>
    <td>ビット S に i 番目のフラグが消えているかどうか</td>
    <td>if (!(S &amp; (1&lt;&lt;i)))</td>
    <td>i∉S かどうか</td>
  </tr>
  <tr>
    <td>ビット S に i 番目のフラグを立てる</td>
    <td>S｜= (1&lt;&lt;i)</td>
    <td>S = S∪ {i}</td>
  </tr>
  <tr>
    <td>ビット S に i 番目のフラグを消す</td>
    <td>S &amp;= ~(1&lt;&lt;i)</td>
    <td>S = S- {i}</td>
  </tr>
  <tr>
    <td>ビット S に何個のフラグが立っているか</td>
    <td>__builtin_popcount(S)</td>
    <td>S の要素数</td>
  </tr>
  <tr>
    <td>すべてのフラグが消えている状態</td>
    <td>0</td>
    <td>空集合 ∅</td>
  </tr>
  <tr>
    <td>すべてのフラグが立っている状態</td>
    <td>(1&lt;&lt;n) - 1</td>
    <td>{0,1,…,n} - 1</td>
  </tr>
  <tr>
    <td>ビット S に i 番目のフラグを立てたもの</td>
    <td>S｜(1&lt;&lt;i)</td>
    <td>S ∪ {i}</td>
  </tr>
  <tr>
    <td>ビット S に i 番目のフラグを消したもの</td>
    <td>S &amp; ~(1&lt;&lt;i)</td>
    <td>S - {i}</td>
  </tr>
  <tr>
    <td></td>
    <td>S &amp; T</td>
    <td>S∩T</td>
  </tr>
  <tr>
    <td></td>
    <td>S｜T</td>
    <td>S∪T</td>
  </tr>
  <tr>
    <td></td>
    <td>if (S &amp; T)</td>
    <td>S∩T≠∅ かどうか</td>
  </tr>
  <tr>
    <td></td>
    <td>if (!(S &amp; T))</td>
    <td>S∩T=∅ かどうか</td>
  </tr>
</tbody>
</table>

やりたいこと,実装,集合で言うと
ビット S に i 番目のフラグが立っているかどうか,if (S & (1<<i)),i∈S かどうか
ビット S に i 番目のフラグが消えているかどうか,if (!(S & (1<<i))),i∉S かどうか
ビット S に i 番目のフラグを立てる,S｜= (1<<i),S = S∪ {i}
ビット S に i 番目のフラグを消す,S &= ~(1<<i),S = SS- {i}
ビット S に何個のフラグが立っているか,__builtin_popcount(S),S の要素数
すべてのフラグが消えている状態,0,空集合 ∅
すべてのフラグが立っている状態,(1<<n) - 1,"{0,1,…,n} - 1"
ビット S に i 番目のフラグを立てたもの,S｜(1<<i),S ∪ {i}
ビット S に i 番目のフラグを消したもの,S & ~(1<<i),S - {i}
,S & T,S∩T


In [ ]:
# bitset を使って bitset 関数を使った方が便利
# set, reset, test, flip, all, any, count が使える

In [33]:
# bit 全探索
# bit 全探索とは、n 個の要素からなる集合 {0,1,…,n−10,1,…,n−1} の部分集合をすべて調べ上げる手法のことです
# 全探索アルゴリズムの一種であり、必ずしも効率が良いわけではなく、
# また全探索アルゴリズムを bit 全探索アルゴリズムに書き直せるわけでもない
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
 
int main() {
    int n = 3;
    // これと
    for (int bit = 0; bit < (1<<n); ++bit){
        cout << bit << endl;
    }

    // これが本質的には同じことをやっている => 全探索とは数え上げること
    for (int tmp = 0; tmp < (1 << 3); tmp++) {
        bitset<3> s(tmp);
        cout << s << endl;
    }
}

Overwriting temp.cpp


In [34]:
!g++ temp.cpp; ./a.out

0
1
2
3
4
5
6
7
000
001
010
011
100
101
110
111


In [ ]:
# 問題 bitset を数字で表せ
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
 
int main() {
    for (int tmp = 0; tmp < (1 << 3); tmp++) {
        bitset<3> s(tmp);
        cout << s.to_ullong();
    }
    cout  << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

01234567


In [ ]:
# 問題 bitset を数字で表せ その2
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

vector<int> toNumVec(bitset<3> bs) {
    vector<int> vec;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) vec.push_back(i);
    }
    return vec;
}

void vecout(vector<int> vec){
    cout << "{";
    bool isFirst = true;
    for (auto item : vec){
        if(!isFirst) cout << ", ";
        cout << item;
        isFirst = false;
    };
    cout << "}" << endl;
}

int main() {
    for (int tmp = 0; tmp < (1 << 3); tmp++) {
        bitset<3> s(tmp);
        vecout(toNumVec(s));
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

{}
{0}
{1}
{0, 1}
{2}
{0, 2}
{1, 2}
{0, 1, 2}


In [ ]:
# コーヒーフレーバーの全組み合わせ
# 部分集合全探索
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

vector<string> toStrVec(bitset<4> bs) {
    vector<string> flavor ={"ミルク","砂糖","シナモン","ウィスキー"};
    vector<string> vec;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) {
            vec.push_back(flavor.at(i));
        }
    }
    return vec;
}

string formatVecStr2Str(vector<string> vec){
    string str = "";
    str +="{";
    bool isFirst = true;
    for (auto item : vec){
        if(!isFirst) str += ", ";
        str += item;
        isFirst = false;
    };
    str += "}";
    return str;
}

int main() {
    const int n = 4;
    for (int tmp = 0; tmp < (1 << n); tmp++) {
        bitset<n> s(tmp);
        cout << tmp << ": " << formatVecStr2Str(toStrVec(s)) << endl;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

0: {}
1: {ミルク}
2: {砂糖}
3: {ミルク, 砂糖}
4: {シナモン}
5: {ミルク, シナモン}
6: {砂糖, シナモン}
7: {ミルク, 砂糖, シナモン}
8: {ウィスキー}
9: {ミルク, ウィスキー}
10: {砂糖, ウィスキー}
11: {ミルク, 砂糖, ウィスキー}
12: {シナモン, ウィスキー}
13: {ミルク, シナモン, ウィスキー}
14: {砂糖, シナモン, ウィスキー}
15: {ミルク, 砂糖, シナモン, ウィスキー}


In [ ]:
# 部分和問題
# n 個の整数 a[0],a[1],…,a[n−1] が与えられる。
# これらの整数から何個かの整数を選んで総和を K とすることができるかどうかを判定せよ。
# 例 n=3 a=(7,2,9) K=11 => Yes
# 例2 n=2 a=(5,7) K=11 => No
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    int n;
    int a[25]; // 余裕をもってサイズ 25 に
    int K;
    cin >> n;
    for (int i = 0; i < n; ++i) cin >> a[i];
    cin >> K;
    bool exist = false;
    for (int bit = 0; bit < (1<<n); ++bit) {
        int sum = 0; // 部分集合の和
        for (int i = 0; i < n; ++i) {
            if (bit & (1<<i)) { 
                sum += a[i];
            }
        }
        if (sum == K) exist = true;
    }
    if (exist) cout << "Yes" << endl;
    else cout << "No" << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; echo 3 7 2 9 11 |./a.out #=> Yes
!g++ temp.cpp; echo 2 5 7 11 |./a.out #=> No

Yes
No


In [37]:
# bitset で書き直す
# 理解のために少し単純化する
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    int n = 3; // 要素数 or bit 数
    int K = 11;  // トータル
    vector<int> a {7,2,9};
    bool exist = false;

    for (int tmp = 0; tmp < (1 << n); tmp++) {
        bitset<3> s(tmp);
        int sum = 0;
        for (int i = 0; i < n; i++){
            if (s.test(i)) sum += a.at(i); // {7,2,9} の全部分集合について和を取る
        }
        if (sum == K) exist = true;
    }

    if (exist) cout << "Yes" << endl;
    else cout << "No" << endl;
}

Overwriting temp.cpp


In [38]:
!g++ temp.cpp; ./a.out #=> Yes

Yes


# いまここ


In [ ]:
# さて、この解法では例えば n=100 となると計算時間的に破綻してしまいます。
# for 文のループ回数が 2n 回あるのですが、
# 2^100 = 1,267,650,600,228,229,401,496,703,205,376
# であり、1秒間で処理できる回数は最大でも 1,000,000,0001,000,000,000 回程度なので到底無理です。

# しかしなんと、この問題は n=100 程度でも解くことのできる解法があります！！！ 
# (しかしこれでもまだ効率よく解けると認められる解法ではないです)。
# 興味のある方は擬多項式時間アルゴリズムについて調べるとともに、以下の記事を読んでいただけたらと思います:

# 典型的な DP (動的計画法) のパターンを整理 Part1 ～ ナップサック DP 編 ～

# 7. 与えられた部分集合の部分集合を列挙
# bit 全探索は {0,1,…,n−1} の部分集合を列挙するアルゴリズムでした。
# 今度は、例えば n=10 として、{2,3,5,72,3,5,7} のように必ずしも 0,1,…,9 の
# すべてが揃っているわけではない集合が与えられて、その部分集合を列挙する方法を考えます。

# これは次のように実現できます:

# int main() {
#     int n = 10;
#     int A = (1<<2) | (1<<3) | (1<<5) | (1<<7); // {A = {2, 3, 5, 7}

#     for(int bit = A; ; bit = (bit-1) & A) {
#         /* ここに処理を書く */


#         // 最後の 0 で break
#         if (!bit) break;
#     }
# }

In [ ]:
# 確かめてみましょう。
# {2,3,5,7} の部分集合を列挙する
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int main() {
    int n = 10;
    int A = (1<<2) | (1<<3) | (1<<5) | (1<<7); // {A = {2, 3, 5, 7}

    for(int bit = A; ; bit = (bit-1) & A) {
        /* ここに処理を書く */


        /* きちんとできていることを確認してみる */
        // bit の表す集合を求める
        vector<int> S;
        for (int i = 0; i < n; ++i) {
            if (bit & (1<<i)) { // i が bit に入るかどうか
                S.push_back(i);
            }
        }

        // bit の表す集合の出力
        cout << bit << ": {";
        for (int i = 0; i < (int)S.size(); ++i) {
            cout << S[i] << " ";
        }
        cout << "}" << endl;

        // 最後の 0 で break
        if (!bit) break;
    }
}

Overwriting temp.cpp


In [ ]:
%%time
!g++ temp.cpp; ./a.out

172: {2 3 5 7 }
168: {3 5 7 }
164: {2 5 7 }
160: {5 7 }
140: {2 3 7 }
136: {3 7 }
132: {2 7 }
128: {7 }
44: {2 3 5 }
40: {3 5 }
36: {2 5 }
32: {5 }
12: {2 3 }
8: {3 }
4: {2 }
0: {}
CPU times: user 22.9 ms, sys: 15.3 ms, total: 38.1 ms
Wall time: 1.53 s


In [ ]:
# 実験 理解用
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
 
int main() {
    const int n = 10;
    int A = (1<<2) | (1<<3) | (1<<5) | (1<<7); // {A = {2, 3, 5, 7}
    bitset<n> s(A);
    cout << A << endl;
    cout << s << endl; //=> 0010101100 : 4+8+32+128 => 172
    cout << endl;
    int count = 0;
    for(int bit = A; ; bit = (bit-1) & A) { // の意味は 2項目を省略すると while true と同じになる
        cout << (bitset<10>)bit << endl ;
        count++;
        if (count == 4) break;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

172
0010101100

0010101100
0010101000
0010100100
0010100000


In [ ]:
# bitset で書き直す
# {2,3,5,7} の部分集合を列挙する
# 4桁の全探索にする 
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

vector<int> toIntVec(bitset<4> bs) {
    vector<int> s = {2,3,5,7};
    vector<int> vec;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) {
            vec.push_back(s.at(i));
        }
    }
    return vec;
}

string formatVecInt2Str(vector<int> vec){
    string str = "";
    str +="{";
    bool isFirst = true;
    for (auto item : vec){
        if(!isFirst) str += ", ";
        str += to_string(item);
        isFirst = false;
    };
    str += "}";
    return str;
}

int toNum(bitset<4> bs){
    vector<int> s = {2,3,5,7};
    int num = 0;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) {
            num+=pow(2, s.at(i));
        }
    }
    return num;
}

int main() {
    const int n = 4;
    // for (int tmp = 0; tmp < (1 << n); tmp++) {
    for (int tmp = (1 << n)-1; tmp >= 0; tmp--) {
        bitset<n> s(tmp);
        cout << toNum(s) << ": " << formatVecInt2Str(toIntVec(s)) << endl;
    }
}

Overwriting temp.cpp


In [ ]:
%%time 
!g++ temp.cpp; ./a.out

172: {2, 3, 5, 7}
168: {3, 5, 7}
164: {2, 5, 7}
160: {5, 7}
140: {2, 3, 7}
136: {3, 7}
132: {2, 7}
128: {7}
44: {2, 3, 5}
40: {3, 5}
36: {2, 5}
32: {5}
12: {2, 3}
8: {3}
4: {2}
0: {}
CPU times: user 24.2 ms, sys: 13.1 ms, total: 37.3 ms
Wall time: 1.53 s


In [ ]:
# 確かにできていますね。こんなものがどこで必要になるのかと疑問に思うかもしれませんが、
# 例えば O(3n) な bit DP を実装するときにこのようなことをしたくなります。
# 
# 8. next_combination
# 今度は n 個の要素の集合 {0,1,…,n−1} の部分集合のうち、
# 要素数が K のもののみを列挙する方法を考えます。これは以下のように実現できます 
# (ただし、以下のコードは K=0 のときには動作しないので、その場合は例外処理する必要があります):

%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

/* next combination */
int next_combination(int sub) {
    int x = sub & -sub, y = sub + x;
    return (((sub & ~y) / x) >> 1) | y;
}

int main() {
    int n = 5;  // {0, 1, 2, 3, 4} の部分集合を考える
    int k = 3;

    int bit = (1<<k)-1;  // bit = {0, 1, 2}
    for (;bit < (1<<n); bit = next_combination(bit)) {
        /* ここに処理を書く */


        /* きちんとできていることを確認してみる */
        // bit の表す集合を求める
        vector<int> S;
        for (int i = 0; i < n; ++i) {
            if (bit & (1<<i)) { // i が bit に入るかどうか
                S.push_back(i);
            }
        }

        // bit の表す集合の出力
        cout << bit << ": {";
        for (int i = 0; i < (int)S.size(); ++i) {
            cout << S[i] << " ";
        }
        cout << "}" << endl;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7: {0 1 2 }
11: {0 1 3 }
13: {0 2 3 }
14: {1 2 3 }
19: {0 1 4 }
21: {0 2 4 }
22: {1 2 4 }
25: {0 3 4 }
26: {1 3 4 }
28: {2 3 4 }


In [ ]:
# bitset で書き直す
# 要素数が K のもののみを列挙する
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

vector<int> toIntVec(bitset<5> bs) {
    vector<int> s =  {0, 1, 2, 3, 4};
    vector<int> vec;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) {
            vec.push_back(s.at(i));
        }
    }
    return vec;
}

string formatVecInt2Str(vector<int> vec){
    string str = "";
    str +="{";
    bool isFirst = true;
    for (auto item : vec){
        if(!isFirst) str += ", ";
        str += to_string(item);
        isFirst = false;
    };
    str += "}";
    return str;
}

int toNum(bitset<5> bs){
    vector<int> s =  {0, 1, 2, 3, 4};
    int num = 0;
    for (int i = 0; i < bs.size(); i++){
        if(bs.test(i)) {
            num+=pow(2, s.at(i));
        }
    }
    return num;
}
 
int main() {
    const int n = 5;  // {0, 1, 2, 3, 4} の部分集合を考える
    int k = 3;

    for (int tmp = 0; tmp < (1 << n); tmp++) {
        bitset<n> s(tmp);
        vector<int> vec;
        vec = toIntVec(s);
        if (vec.size() == k) {
            cout << toNum(s) << ": " << formatVecInt2Str(toIntVec(s)) << endl;
        }
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7: {0, 1, 2}
11: {0, 1, 3}
13: {0, 2, 3}
14: {1, 2, 3}
19: {0, 1, 4}
21: {0, 2, 4}
22: {1, 2, 4}
25: {0, 3, 4}
26: {1, 3, 4}
28: {2, 3, 4}


In [ ]:
# 確かにできていますね！ここまで来ると大分マニアックですが、bit 演算は想像以上に色んなことができます。
# これが使えそうな問題として以下があります (使わなくてもできます):

# ABC 018 D バレンタインデー

# 9. Xorshift
# ビットを用いたシンプルな乱数生成方法です。乱数の質が高い割に超高速です！
# C 言語の rand() よりずっと速いです。乱数の質も、メルセンヌツイスターには及ばないものの、
# 2^128−1 の周期を持っていて値の偏りが小さく、実用上十分なことがほとんどです。
# 高速な乱数生成を行いたい際には是非とも使用していきたいところです。

# 参考: Xorshift の論文

# George Marsaglia, 2003: Xorshift RNGs

# 実装もとてつもなくシンプルです。以下に示すようにメイン部分はたった 4 行で書けます！

%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

unsigned int randInt() {
    static unsigned int tx = 123456789, ty=362436069, tz=521288629, tw=88675123;
    unsigned int tt = (tx^(tx<<11));
    tx = ty; ty = tz; tz = tw;
    return ( tw=(tw^(tw>>19))^(tt^(tt>>8)) );
}

int main() {
    // サイコロを 100 回振ってみる
    for (int i = 1; i <= 100; ++i) {
        cout << randInt() % 6 + 1 << ", ";
        if (i % 10 == 0) cout << endl; // 10 回ごとに改行
    }
    return 0;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

3, 1, 1, 3, 1, 5, 4, 3, 1, 2, 
3, 4, 5, 6, 5, 2, 5, 5, 5, 5, 
3, 1, 1, 5, 6, 4, 4, 5, 1, 2, 
2, 1, 5, 3, 4, 2, 3, 1, 4, 6, 
3, 5, 1, 2, 2, 1, 2, 3, 1, 6, 
1, 2, 6, 3, 1, 1, 4, 1, 5, 1, 
3, 6, 1, 5, 2, 1, 1, 3, 6, 1, 
5, 3, 4, 5, 1, 3, 5, 1, 5, 4, 
6, 1, 6, 5, 5, 6, 3, 4, 2, 3, 
2, 1, 3, 6, 1, 6, 1, 2, 1, 6, 


In [ ]:
# get_rand と言うのがあるらしい
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;
int get_rand() {
    return rand();
};
int main() {
    srand(time(0));
    for (int i = 1; i <= 100; ++i) {
        cout << get_rand() % 6 + 1 << ", ";
        if (i % 10 == 0) cout << endl; // 10 回ごとに改行
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

In [ ]:
# 100,000 回サイコロを振ってそれぞれの回数を数えてみます。

%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

unsigned int randInt() {
    static unsigned int tx = 123456789, ty=362436069, tz=521288629, tw=88675123;
    unsigned int tt = (tx^(tx<<11));
    tx = ty; ty = tz; tz = tw;
    return ( tw=(tw^(tw>>19))^(tt^(tt>>8)) );
}

int main() {
    int count[6] = {0};

    for (int i = 0; i < 100000; ++i) {
        int rng = randInt() % 6;
        count[rng]++;
    }

    for (int i = 0; i < 6; ++i) {
        cout << i+1 << ": " << count[i] << " 回" << endl;
    }
}

Writing temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

1: 16602 回
2: 16512 回
3: 16556 回
4: 16689 回
5: 16939 回
6: 16702 回


In [ ]:
# 同じことを get_rand と vector でやってみる
%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

int get_rand() {
    return rand();
};

int main() {
    vector<int> cnt(6);
    srand(time(0));
    for (int i = 1; i <= 100000; ++i) {
        int rng = get_rand()%6;
        cnt.at(rng)++;
    };

    for (int tmp=0;tmp < cnt.size(); ++tmp){
        cout << tmp+1 << ": " << cnt.at(tmp) << " 回" << endl;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

1: 16812 回
2: 16512 回
3: 16733 回
4: 16741 回
5: 16527 回
6: 16675 回


In [ ]:
# かなり均等な結果となっていますね！

# 10. Binary Indexed Tree (BIT)
# ビットを鮮やかに使いこなすデータ構造 BIT です。名前が凄まじくまぎらわしい！！！！！
# BIT は区間に対する処理を高速に実現できるようになるデータ構造です。
# 以下のようなクエリが大量に飛んでくる状況を考えてみましょう (a や b や v の値はクエリ毎に変わる想定です):

# 配列の a 番目から b 番目までに v を加算する
# 配列の a 番目から b 番目までの総和を求める

# 普通にやるとこれらはそれぞれ b - a + 1 箇所にアクセスすることになります。
# もし配列が巨大な場合、このようなクエリが大量に飛んで来られると途方もない時間がかかることになります。
# ビックデータという言葉が至るところに飛び交う昨今、このような大規模データを扱う大量のクエリを
# 高速に実施していくタイプの要求はひたすらに高まっています。

# そしてなんと、BIT を用いると、上のクエリをともに超高速に実現することができます！！！
# 具体的には配列サイズを NN とすると、普通にやったら O(N)O(N) かかるクエリを、O(logN)O(log⁡N) 
# で実施することができます。ここでは詳しい解説は省略して、以下の資料を参照していただけたらと思います:

# Binary Indexed Tree のはなし (非常によくまとまっています)

# Peter M. Fenwick, 1994: A New Data Structure for Cumulative Frequency Tables (元論文です)

# 11. bit DP
# 最後にここまでに登場したビット演算技法を用いて bit DP な処理をサッと書いてみましょう。
# bit DP は、n個の要素の順列 (n! 通りあります) としてありうるものの中から最適なものを
# 求めたい場面でしばしば使えるテクニックです。bit DP は以下のような動的計画法フレームワークです:

# dp[S] := 全体集合 {0,1,…,n−1} の部分集合 S について、

# その中で順序を最適化したときの、S の中での最小コスト

# bit DP フレームワークでは、以下のような漸化式が成立することが多いです:

# dp[S] = min_{i∈S} (dp[S - {i}] + cost(S \ {i}, i))

# この式の意味を簡単に解説します。

# S についてその中で順序を最適化したときのコスト dp[S] を計算したいのですが、
# その順序の最後の要素がなんだったかで場合分けします:

# S の最後が i だったとき、S の最後の i を除いた部分集合についての最適解は 
# dp[S - {i}] になります。S - {i} に i を加えて増えるコストを cost(S \ {i}, i) とすると、
# 結局最小コストは dp[S - {i}] + cost(S - {i}, i) になります。

# あとは i を i∈S の範囲で全部試してあげて、総合的に一番コストが小さかったものを採用します。
# 以上をまとめると bit DP フレームワークは以下のように表せます:

# ＜bit DP 遷移式＞

# dp[S] = min_{i∈S} (dp[S - {i}] + cost(S - {i}, i))

# ＜初期値＞

# dp[0] = 0 (何もない初期状態)

# ＜最終的な最小コスト＞

# dp[(1<<n) - 1]
# 　

# bit DP フレームワークを適用できる具体的な問題として、これまた組合せ最適化の分野では
# 非常に有名な問題である巡回セールスマン問題を考えてみます。
# この問題もまた、効率よく解けるアルゴリズムは存在しないと広く信じられている NP 困難問題であり、
# 万が一効率よく解けるアルゴリズムを開発できたらノーベル賞級です。

# 巡回セールスマン問題

# n 個の都市を、好きな都市から出発してちょうど一度ずつすべての都市を訪れたい。
# 一度訪れた都市にはもう一度行くことはできない。所要時間を最短になるようにせよ。
# なお、街 i から街 j へと移動するのにかかる時間 dist[i][j] が与えられる。

# 【制約】
# ・1≤n≤20
# ・0≤dist[i][j]≤1000
# ・dist[i][i]=0
# ・dist[i][j]=dist[j][i]

# 【数値例】
# 1)
# 　n=3
# 　dist=
# 0 5 2
# 5 0 4
# 2 4 0
# 　答え: 6 (0 -> 2 -> 1 の順にたどると 6 秒で行けます)

# 2)
# 　n=4
# 　dist=
# 0 8 7 3
# 8 0 9 1
# 7 9 0 4
# 3 1 4 0
# 　答え: 11 (1 -> 3 -> 0 -> 2 の順にたどると 11 秒で行けます)


In [ ]:
# まさに、n 個の都市を巡る順序 (n! 通りあります) を最適化する問題ですね。bit DP フレームワークに乗せて

# dp[S] := 全体集合 {0,1,…,n−1} の部分集合 S について、その中で順序を最適化したときの、S の中での最小コスト
# としたくなるのですが、今回は少しだけ工夫が必要です。bit DP の漸化式を見ると

# dp[S] = min_{i∈S} (dp[S - {i}] + cost(S - {i}, i))

# となっており、cost(S - {i}, i) を計算できる必要があります。しかしこれは、S - {i} のうち、
# どの都市が最後だったかに依存 (最後の都市が u なら dist[u][i] です) してしまうため、
# このままでは一つに決まりません。そこで次のように DP テーブルを修正します:

# dp[S][v] := 全体集合 {0,1,…,n−1} の部分集合 S について、
# 最後が v であるという制約の下で順序を最適化したときの、S の中での最小コスト
# DP 漸化式の方も少々の修正で実現できます:

# dp[S][v] = min_{u∈(S−{v})} (dp[S - {v}][u] + dist[u][v])

# 今度は S の最後は v だとわかっているため、S - {v} の最後が何かで場合分けしています。
# また初期値と最終的な値は次のようになります:

# ＜初期値＞

# dp[1<<v][v]=0 (v からスタート)

# ＜最終的な最小コスト＞

# min_v dp[(1<<n) - 1][v]

# 以上を実装すると以下のようになります。今回はメモ化再帰の形式で実装してみます。

%%writefile temp.cpp
#include <bits/stdc++.h>
using namespace std;

const int INF = 100000000; // 十分大きな値

/* 入力 */
int N;
int dist[21][21];

/* メモ再帰 */
int dp[(1<<20) + 1][21]; // dpテーブルは余裕をもったサイズにする
int rec(int bit, int v)
{
    // すでに探索済みだったらリターン
    if (dp[bit][v] != -1) return dp[bit][v];

    // 初期値
    if (bit == (1<<v)) {
        return dp[bit][v] = 0;
    }

    // 答えを格納する変数
    int res = INF;

    // bit の v を除いたもの
    int prev_bit = bit & ~(1<<v);

    // v の手前のノードとして u を全探索
    for (int u = 0; u < N; ++u) {
        if (!(prev_bit & (1<<u))) continue; // u が prev_bit になかったらダメ

        // 再帰的に探索
        if (res > rec(prev_bit, u) + dist[u][v]) {
            res = rec(prev_bit, u) + dist[u][v];
        }
    }

    return dp[bit][v] = res; // メモしながらリターン
}

int main()
{
    // 入力
    cin >> N;
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            cin >> dist[i][j];
        }
    }

    // テーブルを全部 -1 にしておく (-1 でなかったところは探索済)
    for (int bit = 0; bit < (1<<N); ++bit) for (int v = 0; v < N; ++v) dp[bit][v] = -1;

    // 探索
    int res = INF;
    for (int v = 0; v < N; ++v) {
        if (res > rec((1<<N)-1, v)) {
            res = rec((1<<N)-1, v);
        }
    }
    cout << res << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; echo 3 0 5 2 5 0 4 2 4 0 |./a.out #=> 6
!g++ temp.cpp; echo 4 0 8 7 3 8 0 9 1 7 9 0 4 3 1 4 0 |./a.out #=> 11

6
11


In [ ]:
# bit DP による計算時間の改善効果

# bit DP は、n 個の要素の順列としてありうるものの中から最適なものを求めたい場面で使える
# フレームワークでした。計算時間の改善は、O(n!)O から O(n2^n) になります
#  (巡回セールスマン問題の場合は O(n^2 2^n))。

# 計算時間を改善しても結局指数時間なので、あまり有効でない雰囲気も漂わせていますが、O(n!) な
# 全探索だと n≤10 程度までしか実施できない探索が、n≤20 程度までできるようになります。
%%html
<table border="1">
<thead>
<tr><th>計算量</th><th>n=1</th><th>n=5</th><th>n=10</th><th>n=15</th><th>n=20</th><th>n=25</th><th>n=30</th></tr>
</thead>
<tbody>
<tr><td>O(n2)</td><td>1　 　</td><td>25</td><td>100</td><td>225</td><td>400</td><td>625</td><td>900</td></tr>
<tr><td>O(n2n)</td><td>2 　　</td><td>160</td><td>10,240</td><td>491,520</td><td>20,971,520</td><td>838,860,800</td><td>32,212,254,720</td></tr>
<tr><td>O(n!)</td><td>1 　　</td><td>120</td><td>3,628,800</td><td>1,307,674,368,000</td><td>-</td><td>-</td><td>-</td></tr>
</tbody>
</table>


計算量,n=1,n=5,n=10,n=15,n=20,n=25,n=30
O(n2),1,25,100,225,400,625,900
O(n2n),2,160,"10,240","491,520","20,971,520","838,860,800","32,212,254,720"
O(n!),1,120,"3,628,800","1,307,674,368,000",-,-,-


In [ ]:
この延命のような n の伸びがクリティカルに効いてくる場面も少なくありません。
私自身も実務案件で bitDP が有効に利用できた場面が過去二度ほどありました。

付録: next_combination の成立理由

今まで出て来た例のうち、next_combination はなぜあのような実装で実現できるのかがあまり
自明ではありません。詳しく知りたい方はプログラミングコンテストチャレンジブックの 
P.143〜145 のコラムを読んでいただければと思います。

おおまかに以下のような性質を利用しています:

bit の最も右の 1 のみを抽出する: nbit = bit & -bit　(BIT でもおなじみの処理です)
bit の最も右の 1 を 0 にする: nbit = bit & (bit - 1)

-1. おわりに
ビットが活用できる場面について、典型的なものから高度なアルゴリズムまで幅広く特集してみました。
本特集がどこかでなんらかの課題の解決に寄与できたならば、とても嬉しく思います。

drken
けんちょん (Otsuki)


# いまここ